## 1. Setup

In [1]:
!mkdir ./packages

In [2]:
# Copy CLIP-Interrogator files to working directory
!cp -r /kaggle/input/stable-diffusion-image-to-prompts-dataset/ci-preprocess ./

# Copy packages to current working directory
!cp -r /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages/LAVIS-main/LAVIS-main ./packages
!cp -r /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages/clip-interrogator-0.6.0/clip-interrogator-0.6.0 ./packages
!cp -r /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages/safetensors-0.3.0/safetensors-0.3.0 ./packages
!cp -r /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages/sacremoses-0.0.53/sacremoses-0.0.53 ./packages
!cp -r /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages/antlr4-python3-runtime-4.9.3/antlr4-python3-runtime-4.9.3 ./packages
!cp -r /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages/contexttimer-0.3.3/contexttimer-0.3.3 ./packages
!cp -r /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages/fairscale-0.4.4/fairscale-0.4.4 ./packages
!cp -r /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages/iopath-0.1.10/iopath-0.1.10 ./packages
!cp -r /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages/pycocotools-2.0.6/pycocotools-2.0.6 ./packages
!cp -r /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages/validators-0.20.0/validators-0.20.0 ./packages

In [3]:
import sys
sys.path.append('/kaggle/input/stable-diffusion-image-to-prompts-dataset/packages/sentence-transformers-2.2.2')

In [4]:
INSTALL_OFA = False

if INSTALL_OFA:
    !pip install ./packages/sacremoses-0.0.53 --no-index --find-links /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages
    !pip install /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages/transformers-4.18.0.dev0-py3-none-any.whl --no-index --find-links /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages
    
    from transformers import OFAModel, OFATokenizer

In [5]:
INSTALL_LAVIS = False

if INSTALL_LAVIS:
    !pip install ./packages/antlr4-python3-runtime-4.9.3
    !pip install ./packages/contexttimer-0.3.3
    !pip install ./packages/fairscale-0.4.4 --find-links /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages
    !pip install ./packages/iopath-0.1.10
    !pip install ./packages/pycocotools-2.0.6 --find-links /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages
    !pip install ./packages/validators-0.20.0
    !pip install ./packages/LAVIS-main --no-index --find-links /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages
    
    from lavis.models import load_model_and_preprocess, registry

In [6]:
'''
%%writefile /opt/conda/lib/python3.7/site-packages/lavis/models/blip_models/blip.py

"""
 Copyright (c) 2022, salesforce.com, inc.
 All rights reserved.
 SPDX-License-Identifier: BSD-3-Clause
 For full license text, see the LICENSE file in the repo root or https://opensource.org/licenses/BSD-3-Clause
"""

import logging
import os

import torch
from lavis.common.dist_utils import download_cached_file
from lavis.common.utils import is_url
from lavis.models.base_model import BaseModel
from lavis.models.vit import interpolate_pos_embed
from transformers import BertTokenizer


class BlipBase(BaseModel):
    @classmethod
    def init_tokenizer(cls):
        tokenizer = BertTokenizer.from_pretrained("/kaggle/input/huggingface-bert/bert-base-uncased")
        tokenizer.add_special_tokens({"bos_token": "[DEC]"})
        tokenizer.add_special_tokens({"additional_special_tokens": ["[ENC]"]})
        tokenizer.enc_token_id = tokenizer.additional_special_tokens_ids[0]
        return tokenizer

    def load_from_pretrained(self, url_or_filename):
        if is_url(url_or_filename):
            cached_file = download_cached_file(
                url_or_filename, check_hash=False, progress=True
            )
            checkpoint = torch.load(cached_file, map_location="cpu")
        elif os.path.isfile(url_or_filename):
            checkpoint = torch.load(url_or_filename, map_location="cpu")
        else:
            raise RuntimeError("checkpoint url or path is invalid")

        state_dict = checkpoint["model"]

        state_dict["visual_encoder.pos_embed"] = interpolate_pos_embed(
            state_dict["visual_encoder.pos_embed"], self.visual_encoder
        )
        if "visual_encoder_m.pos_embed" in self.state_dict().keys():
            state_dict["visual_encoder_m.pos_embed"] = interpolate_pos_embed(
                state_dict["visual_encoder_m.pos_embed"], self.visual_encoder_m
            )

        for key in self.state_dict().keys():
            if key in state_dict.keys():
                if state_dict[key].shape != self.state_dict()[key].shape:
                    del state_dict[key]

        msg = self.load_state_dict(state_dict, strict=False)

        logging.info("Missing keys {}".format(msg.missing_keys))
        logging.info("load checkpoint from %s" % url_or_filename)

        return msg
'''

'\n%%writefile /opt/conda/lib/python3.7/site-packages/lavis/models/blip_models/blip.py\n\n"""\n Copyright (c) 2022, salesforce.com, inc.\n All rights reserved.\n SPDX-License-Identifier: BSD-3-Clause\n For full license text, see the LICENSE file in the repo root or https://opensource.org/licenses/BSD-3-Clause\n"""\n\nimport logging\nimport os\n\nimport torch\nfrom lavis.common.dist_utils import download_cached_file\nfrom lavis.common.utils import is_url\nfrom lavis.models.base_model import BaseModel\nfrom lavis.models.vit import interpolate_pos_embed\nfrom transformers import BertTokenizer\n\n\nclass BlipBase(BaseModel):\n    @classmethod\n    def init_tokenizer(cls):\n        tokenizer = BertTokenizer.from_pretrained("/kaggle/input/huggingface-bert/bert-base-uncased")\n        tokenizer.add_special_tokens({"bos_token": "[DEC]"})\n        tokenizer.add_special_tokens({"additional_special_tokens": ["[ENC]"]})\n        tokenizer.enc_token_id = tokenizer.additional_special_tokens_ids[0]\n

In [7]:
!pip install ./packages/clip-interrogator-0.6.0 --no-index --find-links /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages/

Looking in links: /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages/
Processing ./packages/clip-interrogator-0.6.0
  Installing build dependencies ... - \ | done
  Getting requirements to build wheel ... - done
  Installing backend dependencies ... - \ done
  Preparing metadata (pyproject.toml) ... - done
Processing /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages/safetensors-0.3.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages/open_clip_torch-2.16.0-py3-none-any.whl
Processing /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages/transformers-4.28.1-py3-none-any.whl
Processing /kaggle/input/stable-diffusion-image-to-prompts-dataset/packages/ftfy-6.1.1-py3-none-any.whl
  Created wheel for clip-interrogator: filename=clip_interrogator-0.6.0-py3-none-any.whl size=787803 sha256=bee212aeb36ca9f27ed10b029b9b1b03f155d6fdc72606a327630a67d23b58c3
  Stor

In [8]:
import os
import random
import yaml
import gc
from shutil import copy2
from glob import glob
from pathlib import Path
from tqdm import tqdm

import numpy as np
import pandas as pd
from PIL import Image
import cv2

from sklearn.preprocessing import normalize
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, SequentialSampler
import timm
from transformers import BlipForConditionalGeneration, BlipProcessor
from sentence_transformers import SentenceTransformer
import open_clip
from clip_interrogator import *
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [9]:
competition_dataset = Path('/kaggle/input/stable-diffusion-image-to-prompts')
external_dataset = Path('/kaggle/input/stable-diffusion-image-to-prompts-dataset')

In [10]:
image_paths = glob(str(competition_dataset / 'images' / '*.png'))
image_ids = [image_path.split('/')[-1][:-4] for image_path in image_paths]
df_test_embeddings = pd.DataFrame(dict(imgId_eId=[id + '_' + str(i) for id in image_ids for i in range(384)]))
df_test = pd.DataFrame(image_ids, columns=['image_id'])

print(f'Test Shape: {df_test.shape} - Test Embeddings Shape: {df_test_embeddings.shape}')

Test Shape: (7, 1) - Test Embeddings Shape: (2688, 1)


## 2. Utilities

In [11]:
def set_seed(seed, deterministic_cudnn=False):

    """
    Set random seed for reproducible results

    Parameters
    ----------
    seed: int
        Random seed

    deterministic_cudnn: bool
        Whether to set deterministic cuDNN or not
    """

    if deterministic_cudnn:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def load_model(model_directory_path):
    
    """
    Load models and config file from the given directory path

    Parameters
    ----------
    model_directory_path: str
        Path of the model directory
    
    Returns
    -------
    config: dict
        Dictionary of model configurations
    
    model: torch.nn.Module
        Model
    """
    
    config = yaml.load(open(f'{model_directory_path}/config.yaml', 'r'), Loader=yaml.FullLoader)
    config['model']['model_args']['pretrained'] = False
    model_path = glob(f'{model_directory_path}/*.pt')[0]
        
    model = eval(config['model']['model_class'])(**config['model']['model_args'])
    model.load_state_dict(torch.load(model_path))
    model = model.to(config['training']['device'])
    model.eval()
        
    return config, model


## 3. Datasets

In [12]:
class ImageCaptionDataset(Dataset):

    def __init__(self, image_paths, captions, transforms=None, image_reader='opencv'):

        self.image_paths = image_paths
        self.captions = captions
        self.transforms = transforms
        self.image_reader = image_reader

    def __len__(self):

        """
        Get the length the dataset

        Returns
        -------
        length: int
            Length of the dataset
        """

        return len(self.image_paths)

    def __getitem__(self, idx):

        """
        Get the idxth element in the dataset

        Parameters
        ----------
        idx: int
            Index of the sample (0 <= idx < length of the dataset)

        Returns
        -------
        images: torch.FloatTensor of shape (channel, height, width) or (height, width, channel)
            Image tensor

        captions: torch.Tensor of shape (1)
            Caption tensor
        """

        if self.image_reader == 'pil':
            image = Image.open(self.image_paths[idx]).convert('RGB')
        elif self.image_reader == 'opencv':
            image = cv2.imread(str(self.image_paths[idx]))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        else:
            raise ValueError(f'Invalid image_reader: {self.image_reader}')

        if self.transforms is not None:
            if isinstance(self.transforms, A.Compose):
                image = self.transforms(image=np.array(image))['image'].float()
            elif isinstance(self.transforms, BlipProcessor):
                image = self.transforms(image, 'a photography of', return_tensors='pt').pixel_values
                image = torch.squeeze(image, dim=0)
            else:
                image = self.transforms(image).float()

        if self.captions is not None:
            captions = torch.as_tensor(self.captions[idx])
            return image, captions
        else:
            return image


In [13]:
class ImageEmbeddingDataset(Dataset):

    def __init__(self, image_paths, embeddings, transforms=None):

        self.image_paths = image_paths
        self.embeddings = embeddings
        self.transforms = transforms

    def __len__(self):

        """
        Get the length the dataset

        Returns
        -------
        length: int
            Length of the dataset
        """

        return len(self.image_paths)

    def __getitem__(self, idx):

        """
        Get the idxth element in the dataset

        Parameters
        ----------
        idx: int
            Index of the sample (0 <= idx < length of the dataset)

        Returns
        -------
        images: torch.FloatTensor of shape (channel, height, width) or (height, width, channel)
            Image tensor or array

        embeddings: torch.FloatTensor of shape (384)
            Embedding tensor
        """

        image = cv2.imread(str(self.image_paths[idx]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transforms is not None:
            image = self.transforms(image=np.array(image))['image'].float()
        else:
            image = torch.as_tensor(image, dtype=torch.float)

        if self.embeddings is not None:
            embeddings = torch.as_tensor(self.embeddings[idx], dtype=torch.float)
            return image, embeddings
        else:
            return image


## 4. OFA (transformers)

In [14]:
ofa_large_caption_inference = False

In [15]:
def load_ofa(model_directory_path, device):

    """
    Load model and processor

    Parameters
    ----------
    model_directory_path: str
        Path of the model directory

    device: torch.device
        Location of the model

    Returns
    -------
    model: transformers.OFAModel
        Model

    tokenizer: transformers.OFATokenizer
        Tokenizer
    """

    model = transformers.OFAModel.from_pretrained(model_directory_path)
    model.to(device)
    model.eval()

    tokenizer = transformers.OFATokenizer.from_pretrained(model_directory_path)

    return model, tokenizer


def predict_ofa(inputs, model, tokenizer, prompt, num_beams, no_repeat_ngram_size, max_length, min_length, device):

    """
    Predict given inputs with given model

    Parameters
    ----------
    inputs: torch.FloatTensor of shape (batch, channel, height, width) images
        Inputs tensor

    model: transformers.OFAModel
        Model

    tokenizer: transformers.OFATokenizer
        Tokenizer

    prompt: str
        Prompt used for the image captioning

    num_beams: int
        Number of beams in beam search

    no_repeat_ngram_size: int
        N-gram size

    max_length: int
        Maximum length of the generated text
        
    min_length: int
        Minimum length of the generated text

    device: torch.device
        Location of the model

    Returns
    -------
    outputs: list of shape (batch) texts
        Outputs
    """

    input_ids = tokenizer([prompt] * inputs.size(0), return_tensors='pt').input_ids.to(device)

    with torch.no_grad(), torch.autocast(device_type=device.type, dtype=torch.float16):
        outputs = model.generate(
            input_ids,
            patch_images=inputs,
            num_beams=num_beams,
            no_repeat_ngram_size=no_repeat_ngram_size,
            max_length=max_length,
            min_length=min_length
        )

    outputs = outputs.detach().cpu()
    outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return outputs


In [16]:
if ofa_large_caption_inference:

    set_seed(42, deterministic_cudnn=False)
    device = torch.device('cuda')

    ofa_model, ofa_tokenizer = load_ofa(
        model_directory_path=str(external_dataset / 'ofa-large-caption'),
        device=device
    )

    ofa_transforms = A.Compose([
        A.Resize(
            height=480,
            width=480,
            interpolation=cv2.INTER_NEAREST,
            always_apply=True
        ),
        A.Normalize(
            mean=[0.5, 0.5, 0.5],
            std=[0.5, 0.5, 0.5],
            max_pixel_value=255,
            always_apply=True
        ),
        ToTensorV2(always_apply=True)
    ])

    ofa_dataset = ImageCaptionDataset(
        image_paths=image_paths,
        captions=None,
        transforms=ofa_transforms,
        image_reader='opencv'
    )

    ofa_data_loader = DataLoader(
        ofa_dataset,
        batch_size=16,
        sampler=SequentialSampler(ofa_dataset),
        pin_memory=True,
        drop_last=False,
        num_workers=2,
    )
    
    ofa_predictions = []

    for inputs in tqdm(ofa_data_loader):
        inputs = inputs.to(device)
        predictions = predict_ofa(
            inputs=inputs,
            model=ofa_model,
            tokenizer=ofa_tokenizer,
            prompt='what does the image describe?',
            num_beams=3,
            no_repeat_ngram_size=4,
            max_length=25,
            min_length=5,
            device=device
        )
        ofa_predictions.append(predictions)

    ofa_predictions = np.concatenate(ofa_predictions)
    df_test['ofa_predictions'] = ofa_predictions
    
    ofa_model.to('cpu')
    del ofa_model, ofa_tokenizer, ofa_predictions
    del ofa_transforms, ofa_dataset, ofa_data_loader
    torch.cuda.empty_cache()


## 5. CoCa (open_clip)

In [17]:
coca_vit_l_14_laion2B_s13B_b90k_inference = True
mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_inference = True

In [18]:
def load_coca(model_name, model_checkpoint_path, device):

    """
    Load model and processor

    Parameters
    ----------
    model_name: str
        Name of the model

    model_checkpoint_path: str
        Path of the model checkpoint

    device: torch.device
        Location of the model

    Returns
    -------
    model: open_clip.CoCa
        Model

    processor: torchvision.transforms.Compose
        Image processor
    """

    model = open_clip.create_model(model_name=model_name, pretrained=model_checkpoint_path)
    model.to(device)
    model.eval()

    processor = open_clip.image_transform(
        model.visual.image_size,
        is_train=False,
        mean=getattr(model.visual, 'image_mean', None),
        std=getattr(model.visual, 'image_std', None),
    )

    return model, processor


def predict_coca(inputs, model, device):

    """
    Predict given inputs with given model

    Parameters
    ----------
    inputs: torch.FloatTensor of shape (batch, height, width, channel)
        Inputs tensor

    model: transformers.VisionEncoderDecoderModel
        Model

    device: torch.device
        Location of the model

    Returns
    -------
    outputs: torch.Tensor of shape (batch, max_length) or list of shape (batch)
        Outputs
    """

    with torch.no_grad(), torch.autocast(device_type=device.type, dtype=torch.float16):
        outputs = model.generate(inputs)

    outputs = outputs.detach().cpu()
    outputs = [open_clip.decode(output).split('<end_of_text>')[0].replace('<start_of_text>', '').rstrip(' .,') for output in outputs]

    return outputs


In [19]:
if coca_vit_l_14_laion2B_s13B_b90k_inference:

    set_seed(42, deterministic_cudnn=False)
    device = torch.device('cuda')

    coca_model, coca_processor = load_coca(
        model_name='coca_ViT-L-14',
        model_checkpoint_path=str(external_dataset / 'CoCa-ViT-L-14-laion2B-s13B-b90k' / 'open_clip_pytorch_model.bin'),
        device=device,
    )

    coca_dataset = ImageCaptionDataset(
        image_paths=image_paths,
        captions=None,
        transforms=coca_processor,
        image_reader='pil'
    )

    coca_data_loader = DataLoader(
        coca_dataset,
        batch_size=16,
        sampler=SequentialSampler(coca_dataset),
        pin_memory=True,
        drop_last=False,
        num_workers=2,
    )
    
    coca_predictions = []

    for inputs in tqdm(coca_data_loader):
        inputs = inputs.to(device)
        predictions = predict_coca(
            inputs=inputs,
            model=coca_model,
            device=device
        )
        coca_predictions.append(predictions)

    coca_predictions = np.concatenate(coca_predictions)
    df_test['coca_vit_l_14_laion2B_s13B_b90k_prediction'] = coca_predictions
    
    coca_model.to('cpu')
    del coca_model, coca_processor, coca_predictions
    del coca_dataset, coca_data_loader
    torch.cuda.empty_cache()


100%|██████████| 1/1 [00:07<00:00,  7.28s/it]


In [20]:
if mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_inference:

    set_seed(42, deterministic_cudnn=False)
    device = torch.device('cuda')

    coca_model, coca_processor = load_coca(
        model_name='coca_ViT-L-14',
        model_checkpoint_path=str(external_dataset / 'mscoco_finetuned_CoCa-ViT-L-14-laion2B-s13B-b90k' / 'open_clip_pytorch_model.bin'),
        device=device,
    )

    coca_dataset = ImageCaptionDataset(
        image_paths=image_paths,
        captions=None,
        transforms=coca_processor,
        image_reader='pil'
    )

    coca_data_loader = DataLoader(
        coca_dataset,
        batch_size=16,
        sampler=SequentialSampler(coca_dataset),
        pin_memory=True,
        drop_last=False,
        num_workers=2,
    )
    
    coca_predictions = []

    for inputs in tqdm(coca_data_loader):
        inputs = inputs.to(device)
        predictions = predict_coca(
            inputs=inputs,
            model=coca_model,
            device=device
        )
        coca_predictions.append(predictions)

    coca_predictions = np.concatenate(coca_predictions)
    df_test['mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_prediction'] = coca_predictions
    
    coca_model.to('cpu')
    del coca_model, coca_processor, coca_predictions
    del coca_dataset, coca_data_loader
    torch.cuda.empty_cache()


100%|██████████| 1/1 [00:03<00:00,  3.20s/it]


## 6. BLIP (lavis)

In [21]:
blip_caption_base_inference = False

In [22]:
def load_lavis_blip(model_name, model_type, model_config_path, device):

    """
    Load model and processor

    Parameters
    ----------
    model_name: str
        Name of the model

    model_type: str
        Type of the model

    model_config_path: str
        Path of the model checkpoint
        
    device: torch.device
        Location of the model

    Returns
    -------
    model: lavis.models.BlipCaption
        Model

    image_processor: dict
        Image processor

    text_processor: dict
        Text processor
    """
    
    config = yaml.load(open(model_config_path, 'r'), Loader=yaml.FullLoader)
    config['model']['pretrained'] = str(external_dataset / '/'.join(config['model']['pretrained'].split('/')[-2:]))
    config['model']['finetuned'] = str(external_dataset / '/'.join(config['model']['finetuned'].split('/')[-2:]))
    config['model']['med_config_path'] = str(external_dataset / '/'.join(config['model']['med_config_path'].split('/')[-2:]))
    with open(str(model_config_path).split('/')[-1], mode='w') as f:
        yaml.dump(config, f, default_flow_style=False)
    
    copy2(str(model_config_path).split('/')[-1], f'/opt/conda/lib/python3.7/site-packages/lavis/{str(model_config_path).split("/")[-1]}')
    
    model_class = registry.get_model_class(model_name)
    model_class.PRETRAINED_MODEL_CONFIG_DICT[model_type] = str(model_config_path).split('/')[-1]
    model, image_processor, text_processor = load_model_and_preprocess(name=model_name, model_type=model_type)
    
    model.to(device)
    model.eval()

    return model, image_processor, text_processor


def predict_lavis_blip(inputs, model, nucleus_sampling=False, num_beams=3, max_length=25, min_length=5):

    """
    Predict given inputs with given model

    Parameters
    ----------
    inputs: torch.FloatTensor of shape (batch, height, width, channel) images
        Inputs tensor

    model: transformers.BlipForConditionalGeneration
        Model

    nucleus_sampling: bool
        Whether to use nucleus sampling or not

    num_beams: int
        Number of beams in beam search

    max_length: int
        Max length of the generated sequence

    min_length: int
        Min length of the generated sequence

    Returns
    -------
    outputs: list of shape (batch) texts
        Outputs
    """

    with torch.no_grad(), torch.autocast(device_type=device.type, dtype=torch.float16):
        outputs = model.generate(
            samples={'image': inputs},
            use_nucleus_sampling=nucleus_sampling,
            num_beams=num_beams,
            max_length=max_length,
            min_length=min_length,
            top_p=0.9,
            repetition_penalty=1.0,
            num_captions=1
        )

    return outputs


In [23]:
if blip_caption_base_inference:

    set_seed(42, deterministic_cudnn=False)
    device = torch.device('cuda')

    blip_model, blip_image_processor, blip_text_processor = load_lavis_blip(
        model_name='blip_caption',
        model_type='base_coco',
        model_config_path=external_dataset / 'blip-caption-base' / 'blip_caption_base_coco.yaml',
        device=device
    )

    blip_dataset = ImageCaptionDataset(
        image_paths=image_paths,
        captions=None,
        transforms=blip_image_processor['eval'],
        image_reader='pil'
    )

    blip_data_loader = DataLoader(
        blip_dataset,
        batch_size=16,
        sampler=SequentialSampler(blip_dataset),
        pin_memory=True,
        drop_last=False,
        num_workers=2,
    )
    
    blip_predictions = []

    for inputs in tqdm(blip_data_loader):
        inputs = inputs.to(device)
        predictions = predict_lavis_blip(
            inputs=inputs,
            model=blip_model
        )
        blip_predictions.append(predictions)

    blip_predictions = np.concatenate(blip_predictions)
    df_test['blip_caption_base_prediction'] = blip_predictions
    
    blip_model.to('cpu')
    del blip_model, blip_image_processor, blip_text_processor, blip_predictions
    del blip_dataset, blip_data_loader
    torch.cuda.empty_cache()


## 7. BLIP (transformers)

In [24]:
blip_image_captioning_large_inference = True

In [25]:
def load_transformers_blip(model_directory_path, device):

    """
    Load model and processor

    Parameters
    ----------
    model_directory_path: str
        Directory of the model

    device: torch.device
        Location of the model

    Returns
    -------
    model: transformers.BlipForConditionalGeneration
        Model

    processor: transformers.BlipProcessor
        Processor
    """

    model = BlipForConditionalGeneration.from_pretrained(model_directory_path)
    model.to(device)
    model.eval()

    processor = BlipProcessor.from_pretrained(model_directory_path)

    return model, processor


def predict_transformers_blip(inputs, model, processor, device, num_beams, max_length, min_length):

    """
    Predict given inputs with given model

    Parameters
    ----------
    inputs: torch.FloatTensor of shape (batch, height, width, channel) images
        Inputs tensor

    model: transformers.BlipForConditionalGeneration
        Model
        
    processor: transformers.BlipProcessor
        Processor
        
    device: torch.device
        Location of the model

    num_beams: int
        Number of beams in beam search

    max_length: int
        Max length of the generated sequence

    min_length: int
        Min length of the generated sequence

    Returns
    -------
    outputs: list of shape (batch) texts
        Outputs
    """

    with torch.no_grad(), torch.autocast(device_type=device.type, dtype=torch.float16):
        outputs = model.generate(pixel_values=inputs, num_beams=num_beams, max_length=max_length, min_length=min_length)

    outputs = processor.batch_decode(outputs, skip_special_tokens=True)

    return outputs


In [26]:
if blip_image_captioning_large_inference:

    set_seed(42, deterministic_cudnn=False)
    device = torch.device('cuda')

    blip_model, blip_processor = load_transformers_blip(
        model_directory_path=external_dataset / 'blip-image-captioning-large',
        device=device
    )

    blip_dataset = ImageCaptionDataset(
        image_paths=image_paths,
        captions=None,
        transforms=blip_processor,
        image_reader='pil'
    )

    blip_data_loader = DataLoader(
        blip_dataset,
        batch_size=16,
        sampler=SequentialSampler(blip_dataset),
        pin_memory=True,
        drop_last=False,
        num_workers=2,
    )
    
    blip_predictions = []

    for inputs in tqdm(blip_data_loader):
        inputs = inputs.to(device)
        predictions = predict_transformers_blip(
            inputs=inputs,
            model=blip_model,
            processor=blip_processor,
            device=device,
            num_beams=1,
            max_length=25,
            min_length=5
        )
        blip_predictions.append(predictions)

    blip_predictions = np.concatenate(blip_predictions)
    df_test['blip_image_captioning_large_prediction'] = blip_predictions
    
    blip_model.to('cpu')
    del blip_model, blip_processor, blip_predictions
    del blip_dataset, blip_data_loader
    torch.cuda.empty_cache()


100%|██████████| 1/1 [00:04<00:00,  4.53s/it]


## 8. CLIP-Interrogator

In [27]:
coca_vit_l_14_laion2B_s13B_b90k_interrogation = False
mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_interrogation = True
blip_caption_base_interrogation = False
blip_image_captioning_large_interrogation = False

In [28]:
def load_clip(model_name, model_checkpoint_path, device):

    """
    Load model and processor

    Parameters
    ----------
    model_name: str
        Name of the model

    model_checkpoint_path: str
        Path of the model checkpoint

    device: torch.device
        Location of the model

    Returns
    -------
    model: open_clip.CLIP
        Model

    processor: torchvision.transforms.Compose
        Image processor
    """

    model = open_clip.create_model(
        model_name=model_name,
        pretrained=model_checkpoint_path
    )
    model.to(device)
    model.eval()

    processor = open_clip.image_transform(
        model.visual.image_size,
        is_train=False,
        mean=getattr(model.visual, 'image_mean', None),
        std=getattr(model.visual, 'image_std', None),
    )

    return model, processor


In [29]:
class InterrogatorWithoutForward(clip_interrogator.Interrogator):

    def __init__(self, config):

        super(InterrogatorWithoutForward, self).__init__(config)

    def batch_image_to_features(self, inputs, device):

        """
        Extract image features

        Parameters
        ----------
        inputs: torch.FloatTensor of shape (batch, height, width, channel)
            Inputs tensor

        device: torch.device
            Location of the model

        Returns
        -------
        image_features: torch.FloatTensor of shape (batch, features)
            Feature tensor
        """

        with torch.no_grad(), torch.autocast(device_type=device.type, dtype=torch.float16):
            image_features = self.clip_model.encode_image(inputs)

        image_features = image_features / image_features.norm(dim=1).view(-1, 1)
        image_features = image_features.detach().cpu()

        return image_features

    def interrogate_classic_without_forward(self, image_features, max_flavors, caption):

        """
        Classical interrogation with single medium, artist, trending and movement and given number of flavors

        Parameters
        ----------
        image_features: torch.FloatTensor of shape (batch, features)
            Feature tensor

        max_flavors: int
            Maximum number of flavors

        caption: str
            Caption of the image

        Returns
        -------
        caption: str
            Caption after interrogation
        """

        medium = self.mediums.rank(image_features, 1)[0]
        movement = self.movements.rank(image_features, 1)[0]
        flaves = ", ".join(self.flavors.rank(image_features, max_flavors))

        if caption.startswith(medium):
            prompt = f"{caption}, {movement}, {flaves}"
        else:
            prompt = f"{caption}, {medium}, {movement}, {flaves}"

        return clip_interrogator._truncate_to_fit(prompt, self.tokenize)

    def interrogate_fast_without_forward(self, image_features, max_flavors, caption):

        """
        Fast interrogation with given number of flavors, mediums, artists, trendings and movements

        Parameters
        ----------
        image_features: torch.FloatTensor of shape (batch, features)
            Feature tensor

        max_flavors: int
            Maximum number of flavors

        caption: str
            Caption of the image

        Returns
        -------
        caption: str
            Caption after interrogation
        """

        merged = clip_interrogator._merge_tables([self.artists, self.flavors, self.mediums, self.movements, self.trendings], self)
        tops = merged.rank(image_features, max_flavors)

        return clip_interrogator._truncate_to_fit(caption + ", " + ", ".join(tops), self.tokenize)

    def interrogate_negative_without_forward(self, image_features, max_flavors):

        """
        Negative interrogation with given number of flavors

        Parameters
        ----------
        image_features: torch.FloatTensor of shape (batch, features)
            Feature tensor

        max_flavors: int
            Maximum number of flavors

        Returns
        -------
        caption: str
            Caption after interrogation
        """

        flaves = self.flavors.rank(image_features, self.config.flavor_intermediate_count, reverse=True)
        flaves = flaves + self.negative.labels

        return self.chain(image_features, flaves, max_count=max_flavors, reverse=True)

    def interrogate_without_forward(self, image_features, min_flavors, max_flavors, caption):

        """
        Negative interrogation with given number of flavors

        Parameters
        ----------
        image_features: torch.FloatTensor of shape (batch, features)
            Feature tensor

        max_flavors: int
            Maximum number of flavors

        Returns
        -------
        caption: str
            Caption after interrogation
        """

        merged = clip_interrogator._merge_tables([self.artists, self.flavors, self.mediums, self.movements, self.trendings], self)
        flaves = merged.rank(image_features, self.config.flavor_intermediate_count)
        best_prompt, best_sim = caption, self.similarity(image_features, caption)

        best_prompt = self.chain(image_features, flaves, best_prompt, best_sim, min_count=min_flavors, max_count=max_flavors)

        fast_prompt = self.interrogate_fast_without_forward(image_features, max_flavors, caption=caption)
        classic_prompt = self.interrogate_classic_without_forward(image_features, max_flavors, caption=caption)
        candidates = [caption, classic_prompt, fast_prompt, best_prompt]
        
        return candidates[np.argmax(self.similarities(image_features, candidates))]


In [30]:
if coca_vit_l_14_laion2B_s13B_b90k_interrogation or mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_interrogation or blip_caption_base_interrogation or blip_image_captioning_large_interrogation:

    set_seed(42, deterministic_cudnn=False)
    device = torch.device('cuda')

    clip_model, clip_processor = load_clip(
        model_name='ViT-H-14',
        model_checkpoint_path=str(external_dataset / 'CLIP-ViT-H-14-laion2B-s32B-b79K' / 'open_clip_pytorch_model.bin'),
        device=device
    )
    
    clip_dataset = ImageCaptionDataset(
        image_paths=image_paths,
        captions=None,
        transforms=clip_processor,
        image_reader='pil'
    )
    
    clip_data_loader = DataLoader(
        clip_dataset,
        batch_size=32,
        sampler=SequentialSampler(clip_dataset),
        pin_memory=False,
        drop_last=False,
        num_workers=2,
    )
    
    clip_image_features = []

    for inputs in tqdm(clip_data_loader):
        
        inputs = inputs.to(device)
        
        with torch.no_grad(), torch.autocast(device_type=device.type, dtype=torch.float16):
            image_features = clip_model.encode_image(inputs)

        image_features = image_features / image_features.norm(dim=1).view(-1, 1)
        image_features = image_features.detach().cpu()
        clip_image_features.append(image_features)

    clip_image_features = torch.cat(clip_image_features, dim=0)
    clip_model.to('cpu')
    torch.cuda.empty_cache()

    del clip_dataset, clip_data_loader


100%|██████████| 1/1 [00:00<00:00,  1.12it/s]


In [31]:
if coca_vit_l_14_laion2B_s13B_b90k_interrogation or mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_interrogation or blip_caption_base_interrogation or blip_image_captioning_large_interrogation:
    
    ci_config = clip_interrogator.Config()
    ci_config.caption_model = False
    ci_config.caption_processor = False
    ci_config.clip_model = clip_model
    ci_config.clip_preprocess = clip_processor
    ci_config.quiet = True
    ci_config.download_cache = False
    ci_config.clip_model_name = 'ViT-H-14/laion2b_s32b_b79k'
    ci_config.cache_path = './ci-preprocess'
    ci = InterrogatorWithoutForward(ci_config)


In [32]:
if blip_image_captioning_large_interrogation:
    
    for idx, (image_features, caption) in tqdm(
        enumerate(zip(
            clip_image_features,
            df_test['blip_image_captioning_large_prediction'].values
        )),
        total=clip_image_features.shape[0]
    ):
        interrogated_caption = ci.interrogate_classic_without_forward(torch.unsqueeze(image_features, dim=0).to(device), 1, caption)
        df_test.loc[idx, 'interrogated_blip_image_captioning_large_prediction'] = interrogated_caption

if coca_vit_l_14_laion2B_s13B_b90k_interrogation:
    
    for idx, (image_features, caption) in tqdm(
        enumerate(zip(
            clip_image_features,
            df_test['coca_vit_l_14_laion2B_s13B_b90k_prediction'].values
        )),
        total=clip_image_features.shape[0]
    ):
        interrogated_caption = ci.interrogate_classic_without_forward(torch.unsqueeze(image_features, dim=0).to(device), 1, caption)
        df_test.loc[idx, 'interrogated_coca_vit_l_14_laion2B_s13B_b90k_prediction'] = interrogated_caption
        
        
if mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_interrogation:
    
    for idx, (image_features, caption) in tqdm(
        enumerate(zip(
            clip_image_features,
            df_test['mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_prediction'].values
        )),
        total=clip_image_features.shape[0]
    ):
        interrogated_caption = ci.interrogate_classic_without_forward(torch.unsqueeze(image_features, dim=0).to(device), 3, caption)
        df_test.loc[idx, 'interrogated_mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_prediction'] = interrogated_caption

if coca_vit_l_14_laion2B_s13B_b90k_interrogation or mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_interrogation or blip_caption_base_interrogation or blip_image_captioning_large_interrogation:
    del clip_model, clip_processor
    del ci, ci_config

100%|██████████| 7/7 [00:02<00:00,  2.88it/s]


In [33]:
language_model = SentenceTransformer(external_dataset / 'sentence-transformers_all-MiniLM-L6-v2')

coca_vit_l_14_laion2B_s13B_b90k_predictions_embeddings = language_model.encode(df_test['coca_vit_l_14_laion2B_s13B_b90k_prediction'])
df_test_embeddings['coca_vit_l_14_laion2B_s13B_b90k_prediction_embedding'] = coca_vit_l_14_laion2B_s13B_b90k_predictions_embeddings.flatten()

mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_predictions_embeddings = language_model.encode(df_test['mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_prediction'])
df_test_embeddings['mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_prediction_embedding'] = mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_predictions_embeddings.flatten()

interrogated_mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_predictions_embeddings = language_model.encode(df_test['interrogated_mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_prediction'])
df_test_embeddings['interrogated_mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_prediction_embedding'] = interrogated_mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_predictions_embeddings.flatten()

blip_image_captioning_large_predictions_embeddings = language_model.encode(df_test['blip_image_captioning_large_prediction'])
df_test_embeddings['blip_image_captioning_large_prediction_embedding'] = blip_image_captioning_large_predictions_embeddings.flatten()

del language_model

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

## 9. KNN

In [34]:
objective_embedding_paths = []
clip_embedding_paths = []

objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-diffusiondb-14m-part1/all_minilm_l6_v2/*.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-diffusiondb-14m-part2/all_minilm_l6_v2/*.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-diffusiondb-14m-part3/all_minilm_l6_v2/*.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-mscoco/all_minilm_l6_v2/*.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-dataset80k/all_minilm_l6_v2/prompt_embeddings_allminilm_001.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-dataset30k/all_minilm_l6_v2/prompt_embeddings_allminilm_001.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-dataset900k/all_minilm_l6_v2/*.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-conceptual-captions/all_minilm_l6_v2/*.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-sd2gpt2/all_minilm_l6_v2/*.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-sd2hardcode/all_minilm_l6_v2/*.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-chatgpt/all_minilm_l6_v2/*.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0000-0004/all_minilm_l6_v2/*.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0005-0009/all_minilm_l6_v2/*.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0010-0014/all_minilm_l6_v2/*.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0015-0019/all_minilm_l6_v2/*.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0020-0024/all_minilm_l6_v2/*.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0025-0029/all_minilm_l6_v2/*.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0030-0034/all_minilm_l6_v2/*.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0035-0039/all_minilm_l6_v2/*.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0040-0044/all_minilm_l6_v2/*.npy'))
objective_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0045-0049/all_minilm_l6_v2/*.npy'))

clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-diffusiondb-14m-part1/vith14/*.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-diffusiondb-14m-part2/vith14/*.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-diffusiondb-14m-part3/vith14/*.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-mscoco/vith14/*.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-dataset80k/vith14/prompt_embedding_vith14_001.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-dataset30k/vith14/prompt_embedding_vith14_001.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-dataset900k/vith14/*.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-conceptual-captions/vith14/*.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-sd2gpt2/vith14/*.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-sd2hardcode/vith14/*.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-chatgpt/vith14/*.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0000-0004/vith14/*.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0005-0009/vith14/*.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0010-0014/vith14/*.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0015-0019/vith14/*.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0020-0024/vith14/*.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0025-0029/vith14/*.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0030-0034/vith14/*.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0035-0039/vith14/*.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0040-0044/vith14/*.npy'))
clip_embedding_paths += sorted(glob('/kaggle/input/pub-embeddings-laion2b-part0045-0049/vith14/*.npy'))

In [35]:
if clip_image_features.shape[0] == 7:
    objective_embedding_paths = objective_embedding_paths[:5]
    clip_embedding_paths = clip_embedding_paths[:5]

In [36]:
n_neighbors = 125
distance_dim = 15
coef = 0.001

In [37]:
def predict_local_knn(ref_x_embeddings, test_x_embeddings, n_neighbors=100, interval=1000, distance_dim=6, coef=0.001):
    
    ref_x_embeddings = torch.from_numpy(ref_x_embeddings).to('cuda')
    ref_x_embeddings /= ref_x_embeddings.norm(dim=-1, keepdim=True)
    
    n_iter = test_x_embeddings.shape[0] // interval
    if (test_x_embeddings.shape[0] % interval) != 0:
        n_iter += 1
        
    dist_topk_store = []
    idxs_topk_store = []
    weights_store = []
    preds = []
    delta = 0.0001
    
    for i in range(n_iter):
        batch_test_embeddings = torch.from_numpy(
            test_x_embeddings[i * interval:(i + 1) * interval, :].copy()
        ).to('cuda')
        batch_test_embeddings /= batch_test_embeddings.norm(dim=-1, keepdim=True)
        
        dists = 1 - torch.mm(batch_test_embeddings, ref_x_embeddings.T)
        del batch_test_embeddings
        gc.collect()
        
        dist_topk, idxs_topk = torch.topk(dists, n_neighbors, largest=False, dim=-1)
        dist_topk = dist_topk.to(torch.float64)        
        
        weights = 1 / (dist_topk ** distance_dim + delta) * coef
        weights[dist_topk < 0] = delta
        
        dist_topk_store.append(dist_topk.to('cpu').detach().numpy().copy())
        idxs_topk_store.append(idxs_topk.to('cpu').detach().numpy().copy())
        weights_store.append(weights.to('cpu').detach().numpy().copy())
                
        del dists, weights, dist_topk, idxs_topk
        torch.cuda.empty_cache()
        gc.collect()
        
    del ref_x_embeddings
    torch.cuda.empty_cache()
    gc.collect()
    return np.concatenate(dist_topk_store), np.concatenate(idxs_topk_store), np.concatenate(weights_store)


In [38]:
for clip_embedding_idx, clip_embedding_path in enumerate(tqdm(clip_embedding_paths)):

    clip_embeddings = np.load(clip_embedding_path).astype(np.float16)
    
    with torch.no_grad():
        local_dists, local_emb_indecies, local_weights = predict_local_knn(
            clip_embeddings,
            clip_image_features.numpy(),
            n_neighbors=n_neighbors,
            distance_dim=distance_dim,
            coef=coef
        )
        
    local_files = np.zeros(local_dists.shape, dtype=np.int32) + clip_embedding_idx
    
    if clip_embedding_idx == 0:
        global_files = local_files
        global_dists = local_dists
        global_emb_indecies = local_emb_indecies
        global_weights = local_weights
    else:
        global_files = np.concatenate([global_files, local_files], axis=-1)
        global_dists = np.concatenate([global_dists, local_dists], axis=-1)
        global_emb_indecies = np.concatenate([global_emb_indecies, local_emb_indecies], axis=-1)
        global_weights = np.concatenate([global_weights, local_weights], axis=-1)

        unsorted_min_indices = np.argpartition(global_dists, n_neighbors, axis=1)[:, :n_neighbors]

        global_files = np.vstack([global_files[i, unsorted_min_indices[i,:]] for i in range(unsorted_min_indices.shape[0])])
        global_dists = np.vstack([global_dists[i, unsorted_min_indices[i,:]] for i in range(unsorted_min_indices.shape[0])])
        global_emb_indecies = np.vstack([global_emb_indecies[i, unsorted_min_indices[i,:]] for i in range(unsorted_min_indices.shape[0])])
        global_weights = np.vstack([global_weights[i, unsorted_min_indices[i,:]] for i in range(unsorted_min_indices.shape[0])])
    
    gc.collect()


100%|██████████| 5/5 [00:25<00:00,  5.18s/it]


In [39]:
df_knn = pd.DataFrame()
df_knn['file'] = global_files.flatten()
df_knn['file'] = df_knn['file'].apply(lambda x: objective_embedding_paths[x])
df_knn['dist'] = global_dists.flatten()
df_knn['emb_index'] = global_emb_indecies.flatten()
df_knn['test_index'] = np.array([[val] * n_neighbors for val in range(clip_image_features.shape[0])]).flatten()
df_knn['weight'] = global_weights.flatten()
df_knn

,file,dist,emb_index,test_index,weight
0,/kaggle/input/pub-embeddings-diffusiondb-14m-p...,0.746094,55979,0,0.080278
1,/kaggle/input/pub-embeddings-diffusiondb-14m-p...,0.739258,44239,0,0.092054
2,/kaggle/input/pub-embeddings-diffusiondb-14m-p...,0.683594,24001,0,0.291854
3,/kaggle/input/pub-embeddings-diffusiondb-14m-p...,0.683594,24004,0,0.291854
4,/kaggle/input/pub-embeddings-diffusiondb-14m-p...,0.683594,24005,0,0.291854
...,...,...,...,...,...
870,/kaggle/input/pub-embeddings-diffusiondb-14m-p...,0.678711,68350,6,0.323912
871,/kaggle/input/pub-embeddings-diffusiondb-14m-p...,0.678711,68349,6,0.323912
872,/kaggle/input/pub-embeddings-diffusiondb-14m-p...,0.678711,68354,6,0.323912
873,/kaggle/input/pub-embeddings-diffusiondb-14m-p...,0.678711,22053,6,0.323912


In [40]:
test_prompt_embeddings = np.zeros((clip_image_features.shape[0], 384))

for (objective_emb_file, gdf) in tqdm(df_knn.groupby('file')):
    ref_objective_embeddings = np.load(objective_emb_file).astype(np.float16) 
    for _, r in gdf.iterrows():
        test_prompt_embeddings[int(r.test_index), :] += r.weight * ref_objective_embeddings[int(r.emb_index), :]


100%|██████████| 5/5 [00:03<00:00,  1.60it/s]


In [41]:
BS = 1000
num = test_prompt_embeddings.shape[0] // BS

if test_prompt_embeddings.shape[0] % BS != 0:
    num += 1
    
for i in range(num):
    embeddings = test_prompt_embeddings[i * BS:(i + 1) * BS, :]
    embeddings = embeddings / (np.abs(embeddings).max(axis=-1, keepdims=True) + 0.0000001)
    embeddings = normalize(embeddings)
    test_prompt_embeddings[i * BS:(i + 1) * BS, :] = embeddings

df_test_embeddings['knn_prediction'] = test_prompt_embeddings.flatten()

## 10. Representation Learning Models

In [42]:
def create_transformations(**transformation_parameters):

    """
    Create training, validation and test sets transformations

    Parameters
    ----------
    transformation_parameters: dict
        Dictionary of transformation parameters

    Returns
    -------
    transformations: dict
        Transformations for training, validation and test sets
    """

    train_transformations = A.Compose([
        A.Resize(
            height=transformation_parameters['resize_height'],
            width=transformation_parameters['resize_width'],
            interpolation=cv2.INTER_NEAREST,
            always_apply=True
        ),
        A.HorizontalFlip(p=transformation_parameters['horizontal_flip_probability']),
        A.Normalize(
            mean=transformation_parameters['normalize_mean'],
            std=transformation_parameters['normalize_std'],
            max_pixel_value=transformation_parameters['normalize_max_pixel_value'],
            always_apply=True
        ),
        ToTensorV2(always_apply=True)
    ])

    val_transformations = A.Compose([
        A.Resize(
            height=transformation_parameters['resize_height'],
            width=transformation_parameters['resize_width'],
            interpolation=cv2.INTER_NEAREST,
            always_apply=True
        ),
        A.Normalize(
            mean=transformation_parameters['normalize_mean'],
            std=transformation_parameters['normalize_std'],
            max_pixel_value=transformation_parameters['normalize_max_pixel_value'],
            always_apply=True
        ),
        ToTensorV2(always_apply=True)
    ])

    test_transformations = A.Compose([
        A.Resize(
            height=transformation_parameters['resize_height'],
            width=transformation_parameters['resize_width'],
            interpolation=cv2.INTER_NEAREST,
            always_apply=True
        ),
        A.Normalize(
            mean=transformation_parameters['normalize_mean'],
            std=transformation_parameters['normalize_std'],
            max_pixel_value=transformation_parameters['normalize_max_pixel_value'],
            always_apply=True
        ),
        ToTensorV2(always_apply=True)
    ])

    transformations = {'train': train_transformations, 'val': val_transformations, 'test': test_transformations}
    return transformations


In [43]:
class TimmTransformerModel(nn.Module):

    def __init__(self, model_name, pretrained, backbone_args, freeze_parameters, head_args):

        super(TimmTransformerModel, self).__init__()

        self.backbone = timm.create_model(
            model_name=model_name,
            pretrained=pretrained,
            **backbone_args
        )

        if freeze_parameters:
            for parameter in self.backbone.parameters():
                parameter.requires_grad = False

        input_features = self.backbone.get_classifier().in_features
        self.backbone.head = nn.Identity()
        self.head = nn.Linear(in_features=input_features, out_features=head_args['output_dimensions'], bias=True)

    def forward(self, x):

        x = self.backbone(x)
        output = self.head(x)

        return output


In [44]:
swin_base_patch4_window12_384_config, swin_base_patch4_window12_384_model = load_model(
    model_directory_path=str(external_dataset / 'swin_base_patch4_window12_384')
)

swin_base_patch4_window12_384_transformations = create_transformations(**swin_base_patch4_window12_384_config['transformations'])

swin_base_patch4_window12_384_dataset = ImageEmbeddingDataset(
    image_paths=image_paths,
    embeddings=None,
    transforms=swin_base_patch4_window12_384_transformations['test'],
)

swin_base_patch4_window12_384_data_loader = DataLoader(
    swin_base_patch4_window12_384_dataset,
    batch_size=16,
    sampler=SequentialSampler(swin_base_patch4_window12_384_dataset),
    pin_memory=False,
    drop_last=False,
    num_workers=2
)

swin_base_patch4_window12_384_predictions = []

for inputs in tqdm(swin_base_patch4_window12_384_data_loader):
    inputs = inputs.to('cuda')

    with torch.no_grad():
        outputs = swin_base_patch4_window12_384_model(inputs)

    outputs = outputs.detach().cpu()

    horizontal_flipped_inputs = inputs.flip(-1)
    with torch.no_grad():
        horizontal_flipped_outputs = swin_base_patch4_window12_384_model(horizontal_flipped_inputs)
    horizontal_flipped_outputs = horizontal_flipped_outputs.detach().cpu()

    outputs = torch.cat([
        outputs.unsqueeze(dim=0),
        horizontal_flipped_outputs.unsqueeze(dim=0)
    ], dim=0).mean(dim=0)

    swin_base_patch4_window12_384_predictions.append(outputs)

swin_base_patch4_window12_384_predictions = torch.cat(swin_base_patch4_window12_384_predictions, dim=0).numpy()
swin_base_patch4_window12_384_predictions = swin_base_patch4_window12_384_predictions / (np.abs(swin_base_patch4_window12_384_predictions).max(axis=-1, keepdims=True) + 0.0000001)
swin_base_patch4_window12_384_predictions = normalize(swin_base_patch4_window12_384_predictions)
df_test_embeddings['swin_base_patch4_window12_384_prediction'] = swin_base_patch4_window12_384_predictions.flatten()

del swin_base_patch4_window12_384_config, swin_base_patch4_window12_384_model
del swin_base_patch4_window12_384_transformations, swin_base_patch4_window12_384_dataset, swin_base_patch4_window12_384_data_loader
del swin_base_patch4_window12_384_predictions

/opt/conda/lib/python3.7/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 1/1 [00:00<00:00,  1.05it/s]


In [45]:
swin_large_patch4_window7_224_config, swin_large_patch4_window7_224_model = load_model(
    model_directory_path=str(external_dataset / 'swin_large_patch4_window7_224')
)

swin_large_patch4_window7_224_transformations = create_transformations(**swin_large_patch4_window7_224_config['transformations'])

swin_large_patch4_window7_224_dataset = ImageEmbeddingDataset(
    image_paths=image_paths,
    embeddings=None,
    transforms=swin_large_patch4_window7_224_transformations['test'],
)

swin_large_patch4_window7_224_data_loader = DataLoader(
    swin_large_patch4_window7_224_dataset,
    batch_size=16,
    sampler=SequentialSampler(swin_large_patch4_window7_224_dataset),
    pin_memory=False,
    drop_last=False,
    num_workers=2
)

swin_large_patch4_window7_224_predictions = []

for inputs in tqdm(swin_large_patch4_window7_224_data_loader):
    inputs = inputs.to('cuda')

    with torch.no_grad():
        outputs = swin_large_patch4_window7_224_model(inputs)

    outputs = outputs.detach().cpu()

    horizontal_flipped_inputs = inputs.flip(-1)
    with torch.no_grad():
        horizontal_flipped_outputs = swin_large_patch4_window7_224_model(horizontal_flipped_inputs)
    horizontal_flipped_outputs = horizontal_flipped_outputs.detach().cpu()

    outputs = torch.cat([
        outputs.unsqueeze(dim=0),
        horizontal_flipped_outputs.unsqueeze(dim=0)
    ], dim=0).mean(dim=0)

    swin_large_patch4_window7_224_predictions.append(outputs)

swin_large_patch4_window7_224_predictions = torch.cat(swin_large_patch4_window7_224_predictions, dim=0).numpy()
swin_large_patch4_window7_224_predictions = swin_large_patch4_window7_224_predictions / (np.abs(swin_large_patch4_window7_224_predictions).max(axis=-1, keepdims=True) + 0.0000001)
swin_large_patch4_window7_224_predictions = normalize(swin_large_patch4_window7_224_predictions)
df_test_embeddings['swin_large_patch4_window7_224_prediction'] = swin_large_patch4_window7_224_predictions.flatten()

del swin_large_patch4_window7_224_config, swin_large_patch4_window7_224_model
del swin_large_patch4_window7_224_transformations, swin_large_patch4_window7_224_dataset, swin_large_patch4_window7_224_data_loader
del swin_large_patch4_window7_224_predictions

100%|██████████| 1/1 [00:00<00:00,  1.56it/s]


In [46]:
swin_large_patch4_window12_384_config, swin_large_patch4_window12_384_model = load_model(
    model_directory_path=str(external_dataset / 'swin_large_patch4_window12_384')
)

swin_large_patch4_window12_384_transformations = create_transformations(**swin_large_patch4_window12_384_config['transformations'])

swin_large_patch4_window12_384_dataset = ImageEmbeddingDataset(
    image_paths=image_paths,
    embeddings=None,
    transforms=swin_large_patch4_window12_384_transformations['test'],
)

swin_large_patch4_window12_384_data_loader = DataLoader(
    swin_large_patch4_window12_384_dataset,
    batch_size=16,
    sampler=SequentialSampler(swin_large_patch4_window12_384_dataset),
    pin_memory=False,
    drop_last=False,
    num_workers=2
)

swin_large_patch4_window12_384_predictions = []

for inputs in tqdm(swin_large_patch4_window12_384_data_loader):
    inputs = inputs.to('cuda')

    with torch.no_grad():
        outputs = swin_large_patch4_window12_384_model(inputs)

    outputs = outputs.detach().cpu()

    horizontal_flipped_inputs = inputs.flip(-1)
    with torch.no_grad():
        horizontal_flipped_outputs = swin_large_patch4_window12_384_model(horizontal_flipped_inputs)
    horizontal_flipped_outputs = horizontal_flipped_outputs.detach().cpu()

    outputs = torch.cat([
        outputs.unsqueeze(dim=0),
        horizontal_flipped_outputs.unsqueeze(dim=0)
    ], dim=0).mean(dim=0)

    swin_large_patch4_window12_384_predictions.append(outputs)

swin_large_patch4_window12_384_predictions = torch.cat(swin_large_patch4_window12_384_predictions, dim=0).numpy()
swin_large_patch4_window12_384_predictions = swin_large_patch4_window12_384_predictions / (np.abs(swin_large_patch4_window12_384_predictions).max(axis=-1, keepdims=True) + 0.0000001)
swin_large_patch4_window12_384_predictions = normalize(swin_large_patch4_window12_384_predictions)
df_test_embeddings['swin_large_patch4_window12_384_prediction'] = swin_large_patch4_window12_384_predictions.flatten()

del swin_large_patch4_window12_384_config, swin_large_patch4_window12_384_model
del swin_large_patch4_window12_384_transformations, swin_large_patch4_window12_384_dataset, swin_large_patch4_window12_384_data_loader
del swin_large_patch4_window12_384_predictions

100%|██████████| 1/1 [00:01<00:00,  1.04s/it]


In [47]:
beitv2_large_patch16_224_config, beitv2_large_patch16_224_model = load_model(
    model_directory_path=str(external_dataset / 'beitv2_large_patch16_224')
)

beitv2_large_patch16_224_transformations = create_transformations(**beitv2_large_patch16_224_config['transformations'])

beitv2_large_patch16_224_dataset = ImageEmbeddingDataset(
    image_paths=image_paths,
    embeddings=None,
    transforms=beitv2_large_patch16_224_transformations['test'],
)

beitv2_large_patch16_224_data_loader = DataLoader(
    beitv2_large_patch16_224_dataset,
    batch_size=16,
    sampler=SequentialSampler(beitv2_large_patch16_224_dataset),
    pin_memory=False,
    drop_last=False,
    num_workers=2
)

beitv2_large_patch16_224_predictions = []

for inputs in tqdm(beitv2_large_patch16_224_data_loader):
    inputs = inputs.to('cuda')

    with torch.no_grad():
        outputs = beitv2_large_patch16_224_model(inputs)

    outputs = outputs.detach().cpu()

    horizontal_flipped_inputs = inputs.flip(-1)
    with torch.no_grad():
        horizontal_flipped_outputs = beitv2_large_patch16_224_model(horizontal_flipped_inputs)
    horizontal_flipped_outputs = horizontal_flipped_outputs.detach().cpu()

    outputs = torch.cat([
        outputs.unsqueeze(dim=0),
        horizontal_flipped_outputs.unsqueeze(dim=0)
    ], dim=0).mean(dim=0)

    beitv2_large_patch16_224_predictions.append(outputs)

beitv2_large_patch16_224_predictions = torch.cat(beitv2_large_patch16_224_predictions, dim=0).numpy()
beitv2_large_patch16_224_predictions = beitv2_large_patch16_224_predictions / (np.abs(beitv2_large_patch16_224_predictions).max(axis=-1, keepdims=True) + 0.0000001)
beitv2_large_patch16_224_predictions = normalize(beitv2_large_patch16_224_predictions)
df_test_embeddings['beitv2_large_patch16_224_prediction'] = beitv2_large_patch16_224_predictions.flatten()

del beitv2_large_patch16_224_config, beitv2_large_patch16_224_model
del beitv2_large_patch16_224_transformations, beitv2_large_patch16_224_dataset, beitv2_large_patch16_224_data_loader
del beitv2_large_patch16_224_predictions

100%|██████████| 1/1 [00:00<00:00,  1.28it/s]


## 11. Ensemble

In [48]:
df_test_embeddings

,imgId_eId,coca_vit_l_14_laion2B_s13B_b90k_prediction_embedding,mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_prediction_embedding,interrogated_mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_prediction_embedding,blip_image_captioning_large_prediction_embedding,knn_prediction,swin_base_patch4_window12_384_prediction,swin_large_patch4_window7_224_prediction,swin_large_patch4_window12_384_prediction,beitv2_large_patch16_224_prediction
0,f27825b2c_0,-0.116310,0.029225,0.027709,-0.020460,-0.106356,-0.033799,-0.054163,-0.052756,-0.084640
1,f27825b2c_1,0.040138,0.053408,0.067174,0.083382,0.056334,0.043510,0.044165,0.033424,0.049386
2,f27825b2c_2,0.033303,-0.084613,-0.075604,-0.068206,0.009878,-0.007162,0.004634,0.001244,-0.019956
3,f27825b2c_3,-0.044670,0.017963,-0.041972,-0.043487,-0.060045,-0.008922,0.014103,-0.022813,-0.009255
4,f27825b2c_4,-0.060771,-0.075529,-0.067199,-0.005878,-0.067769,-0.024105,-0.047492,-0.039126,-0.068727
...,...,...,...,...,...,...,...,...,...,...
2683,c98f79f71_379,-0.013048,-0.073867,-0.072286,-0.049304,-0.030339,0.025875,0.015397,0.016148,0.025705
2684,c98f79f71_380,-0.016317,0.012851,0.044618,0.022202,0.125923,0.058460,0.036276,0.012421,0.069896
2685,c98f79f71_381,0.019852,0.028804,0.056895,0.039194,-0.003771,0.036421,0.049093,0.058329,0.048571
2686,c98f79f71_382,-0.074756,0.009999,-0.059861,-0.032646,-0.020211,-0.029689,-0.026619,-0.031885,-0.037808


In [49]:
df_test_embeddings['val'] =\
(df_test_embeddings['blip_image_captioning_large_prediction_embedding'] * 0.04) +\
(df_test_embeddings['coca_vit_l_14_laion2B_s13B_b90k_prediction_embedding'] * 0.03) +\
(df_test_embeddings['mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_prediction_embedding'] * 0.035) +\
(df_test_embeddings['interrogated_mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_prediction_embedding'] * 0.045) +\
(df_test_embeddings['knn_prediction'] * 0.475) +\
(df_test_embeddings['swin_base_patch4_window12_384_prediction'] * 0.08) +\
(df_test_embeddings['swin_large_patch4_window7_224_prediction'] * 0.09) +\
(df_test_embeddings['swin_large_patch4_window12_384_prediction'] * 0.125) +\
(df_test_embeddings['beitv2_large_patch16_224_prediction'] * 0.08)

df_test_embeddings

,imgId_eId,coca_vit_l_14_laion2B_s13B_b90k_prediction_embedding,mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_prediction_embedding,interrogated_mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_prediction_embedding,blip_image_captioning_large_prediction_embedding,knn_prediction,swin_base_patch4_window12_384_prediction,swin_large_patch4_window7_224_prediction,swin_large_patch4_window12_384_prediction,beitv2_large_patch16_224_prediction,val
0,f27825b2c_0,-0.116310,0.029225,0.027709,-0.020460,-0.106356,-0.033799,-0.054163,-0.052756,-0.084640,-0.073501
1,f27825b2c_1,0.040138,0.053408,0.067174,0.083382,0.056334,0.043510,0.044165,0.033424,0.049386,0.051774
2,f27825b2c_2,0.033303,-0.084613,-0.075604,-0.068206,0.009878,-0.007162,0.004634,0.001244,-0.019956,-0.004998
3,f27825b2c_3,-0.044670,0.017963,-0.041972,-0.043487,-0.060045,-0.008922,0.014103,-0.022813,-0.009255,-0.035898
4,f27825b2c_4,-0.060771,-0.075529,-0.067199,-0.005878,-0.067769,-0.024105,-0.047492,-0.039126,-0.068727,-0.056508
...,...,...,...,...,...,...,...,...,...,...,...
2683,c98f79f71_379,-0.013048,-0.073867,-0.072286,-0.049304,-0.030339,0.025875,0.015397,0.016148,0.025705,-0.015082
2684,c98f79f71_380,-0.016317,0.012851,0.044618,0.022202,0.125923,0.058460,0.036276,0.012421,0.069896,0.077756
2685,c98f79f71_381,0.019852,0.028804,0.056895,0.039194,-0.003771,0.036421,0.049093,0.058329,0.048571,0.022449
2686,c98f79f71_382,-0.074756,0.009999,-0.059861,-0.032646,-0.020211,-0.029689,-0.026619,-0.031885,-0.037808,-0.027273


In [50]:
df_test_embeddings.corr()

,coca_vit_l_14_laion2B_s13B_b90k_prediction_embedding,mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_prediction_embedding,interrogated_mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_prediction_embedding,blip_image_captioning_large_prediction_embedding,knn_prediction,swin_base_patch4_window12_384_prediction,swin_large_patch4_window7_224_prediction,swin_large_patch4_window12_384_prediction,beitv2_large_patch16_224_prediction,val
coca_vit_l_14_laion2B_s13B_b90k_prediction_embedding,1.000000,0.394830,0.415936,0.438780,0.496418,0.503803,0.487945,0.528896,0.491641,0.568876
mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_prediction_embedding,0.394830,1.000000,0.777235,0.562500,0.498671,0.536388,0.541998,0.560076,0.548504,0.615261
interrogated_mscoco_finetuned_coca_vit_l_14_laion2B_s13B_b90k_prediction_embedding,0.415936,0.777235,1.000000,0.543048,0.626302,0.624522,0.627504,0.636011,0.645771,0.721854
blip_image_captioning_large_prediction_embedding,0.438780,0.562500,0.543048,1.000000,0.507106,0.539150,0.534438,0.552370,0.541689,0.609641
knn_prediction,0.496418,0.498671,0.626302,0.507106,1.000000,0.779868,0.778143,0.766443,0.774414,0.951969
swin_base_patch4_window12_384_prediction,0.503803,0.536388,0.624522,0.539150,0.779868,1.000000,0.939095,0.944211,0.919918,0.911615
swin_large_patch4_window7_224_prediction,0.487945,0.541998,0.627504,0.534438,0.778143,0.939095,1.000000,0.941544,0.933726,0.911871
swin_large_patch4_window12_384_prediction,0.528896,0.560076,0.636011,0.552370,0.766443,0.944211,0.941544,1.000000,0.908267,0.909420
beitv2_large_patch16_224_prediction,0.491641,0.548504,0.645771,0.541689,0.774414,0.919918,0.933726,0.908267,1.000000,0.904357
val,0.568876,0.615261,0.721854,0.609641,0.951969,0.911615,0.911871,0.909420,0.904357,1.000000


## 12. Submission

In [51]:
df_test_embeddings[['imgId_eId', 'val']].to_csv('submission.csv', index=False)

In [52]:
!rm -rf /kaggle/working/packages
!rm -rf /kaggle/working/ci-preprocess